In [3]:
import molli as ml
from molli.math import rotation_matrix_from_vectors
import numpy as np
from tqdm import tqdm

def set_origin(conf:ml.ConformerEnsemble,i: int):
    conf.translate(-1*conf.coords[i])
    conf.align_to_ref_coords

def rot_xy(conf: ml.Conformer, q1a:ml.Atom, c0:ml.Atom, c1:ml.Atom):
    '''
    Q1
      \ 
        Q1C = Q2C
    
    This should be positive with respect to right hand rule
    '''
    v1 = conf.get_atom_coord(c0)-conf.get_atom_coord(q1a)
    v2 = conf.get_atom_coord(c1) - conf.get_atom_coord(c0)
    c = np.cross(v1,v2)
    t_matrix = rotation_matrix_from_vectors(c, np.array([0,0,1]))
    conf.transform(t_matrix)

def rot_rz(conf: ml.Conformer, c0:ml.Atom, c1: ml.Atom):
    v1 = conf.get_atom_coord(c1) - conf.get_atom_coord(c0)
    t_matrix = rotation_matrix_from_vectors(v1, np.array([1,0,0]))
    conf.transform(t_matrix)


<>:11: SyntaxWarning: invalid escape sequence '\ '
<>:11: SyntaxWarning: invalid escape sequence '\ '
C:\Users\BlakeOcampo\AppData\Local\Temp\ipykernel_2316\1736734525.py:11: SyntaxWarning: invalid escape sequence '\ '
  '''


In [4]:
clib = ml.ConformerLibrary('6_4_2_DB_CRESTScreen.clib')
align_clib = ml.ConformerLibrary('6_4_3_DB_CRESTScreen_Align.clib', readonly=False, overwrite=True)

with clib.reading(), align_clib.writing():
    for i,k in enumerate(tqdm(clib)):
        ens = clib[k]
        #Isolate Atoms for Alignment
        for a in ens.atoms:
            if 'Q' in a.attrib:
                match a.attrib['Q']:
                    case 1:
                        q1a = a
                    case 2:
                        q2a = a
                    case 3:
                        q3a = a
                    case 4:
                        q4a = a
            if 'C0' in a.attrib:
                c0 = a
            if 'C1' in a.attrib:
                c1 = a

        #Individually Translate Atoms for Alignment
        for conf in ens:
            set_origin(conf,conf.get_atom_index(c0))
            #Rotates molecule such that Q1 and alkene atoms are in the XY plane
            rot_xy(conf,q1a,c0,c1)
            #Rotates molecule such that alkene atoms are along the X axis
            rot_rz(conf,c0,c1)

        align_clib[k] = ens

100%|██████████| 789/789 [08:33<00:00,  1.54it/s]


In [6]:
ml.visual.configure()
ens

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

ConformerEnsemble(name='react_299', formula='C17 H26 O1 Si1')